### 变量管理
在定义inference函数时需要提供NN的所有参数，NN过大时馈送参数很麻烦。TensorFlow 提供了通过变量名称来创建或获取一个变量的机制。通过这个机制可以在不同函数中直接通过变量的名字来使用变量，而不需要将变量以参数的形式到处传递。TensorFlow主要通过tf.get_variable和tf.variable_scope函数实现通过变量名获取变量值。

In [2]:
#下面两个定义等价
v1=tf.Variable(tf.constant(1.0,shape=[1]),name='v')

#提供未读信息与初始化方法，指定变量名称为必须项
v=tf.get_variable("v",shape=[1],initializer=tf.constant_initializer(1.0))


tf.get_variable首先会创建名为“v”的参数，有同名参数会创建失败。tf.get_variable获取一个已创建的变量需要使用tf.variable_scope来控制。

In [3]:
#在名为foo的命名空间内创建名为v的变量
with tf.variable_scope("foo"):
    v=tf.get_variable('v',[1],initializer=tf.constant_initializer(1.0))
    
#因为在命名空间foo中已经存在名字为v的变量，下面的代码将报错:
#with tf.variable_scope("foo"):
#    v=tf.get_variable("v",[1])

#在生成上下文管理器时，将参数reuse设置为Ture。这样tf.get_variable函数将直接获取已经声明的变量
with tf.variable_scope("foo",reuse=True):
    v1=tf.get_variable('v',[1])
    print (v == v1)
    
#将参数reuse设置为ture时，tf.variable_scope将只能获取已经创建的变量。若命名空间foo无该变量，则会报错。

True


In [8]:
#当tf.variable_scope函数嵌套时，reuse参数的取值如何确定

with tf.variable_scope('root'):
    
    #可以通过tf.get_variable_scope().reuse函数获取当前上下文管理器中reuse参数的信息,以下输出False，即最外层reuse是False
    print(tf.get_variable_scope().reuse)
    
    #新建一个嵌套的上下文管理器，并指定reuse为Ture
    with tf.variable_scope('foo',reuse=True):
        #输出True
        print(tf.get_variable_scope().reuse)
        
        #新建一个嵌套的上下文管理器，但不指定reuse，这时reuse的取值会和外面一层保持一致，以下输出True
        with tf.variable_scope('bat'):
            print(tf.get_variable_scope().reuse)
    #退出reuse设置为True的上下文后，又会输出False
    

False
True
True


tf.variable_scope函数生成上下文管理器，也会创建一个TensorFlow中的命名空间，在命名空间中创建的变量名称会带上命名空间名作为前缀。以下展示了如何通过tf.variable_scope管理变量的名称。

In [9]:
v1=tf.get_variable('v',[1])
#“:0”表示这个变量是生成变量这个运算的第一个结果
print(v1.name)

with tf.variable_scope('foo'):
    v2=tf.get_variable('v',[1])
    #在tf.variable_scope中创建的变量，名称前会加入命名空间的名称
    print(v2.name)
    
with tf.variable_scope('foo'):
    with tf.variable_scope('bar'):
        v3=tf.get_variable('v',[1])
        print(v3.name)
        
#创建一个名称为空的命名空间，并设置reuse=True
with tf.variable_scope('',reuse=True):
    #可以直接通过带命名空间名称的变量名来获取其他命名空间下的变量
    v4=tf.get_variable('foo/bar/v',[1])
    print(v4.name)
    print(v4 == v3)

v:0
foo/v:0
foo/bar/v:0
foo/bar/v:0
True


通过变量管理，我们可以将上面定义的推断过程函数做一些改进。

In [2]:
INPUT_NODE=10
LAYER1_NODE=32
OUTPUT_NODE=2

def inference (input_tensor,reuse=False):
    
    #定义第一层神经网络的变量和前向传播过程
    #根据传进来的reuse判断是创建新变量还是使用已经创建好的。没有传入reuse参数时，默认为False.
    with tf.variable_scope('layer1',reuse=reuse):
        weights=tf.get_variable('weights',[INPUT_NODE,LAYER1_NODE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases=tf.get_variable('biases',[LAYER1_NODE],initializer=tf.constant_initializer(0.0))
        layer1=tf.nn.relu(tf.matmul(input_tensor,weights)+biases)
        
    #定义第二层神经网络的变量与前向传播过程，matmul函数注意左乘和右乘的区别
    with tf.variable_scope('layer2',reuse=reuse):
        weights=tf.get_variable('weights',[LAYER1_NODE,OUTPUT_NODE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases=tf.get_variable('biases',[OUTPUT_NODE],initializer=tf.constant_initializer(0.0))
        layer2=tf.matmul(layer1,weights)+biases
    return(layer2)

x=tf.placeholder(tf.float32,[None,INPUT_NODE])
y=inference(x)

#使用训练好的神经网络进行推断时，直接调用inference(new_x,True)。使用上述代码表述1，不需要将所有变量作为参数传递到不同函数中。

#### Tensorflow代码持久化
保存训练好的模型，包括计算图、权重参数等

In [5]:
#保存TensorFlow计算图
import tensorflow as tf

#声明两个变量，并计算它们的和
v1=tf.Variable(tf.constant(1.0,shape=[1]),name='v1')
v2=tf.Variable(tf.constant(2.0,shape=[1]),name='v2')
result=v1+v2

#声明tf.train.Saver()类用于保存模型
saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess,'model/1/model1.ckpt')

#*.ckpt.meta保存了Tensorflow计算图的结构。*.ckpt保存了TensorFlow中每个变量的取值。checkpoint保存了一个目录下所有的模型文件列表

In [1]:
import tensorflow as tf

v1=tf.Variable(tf.constant(22.0,shape=[1]),name='v1')
v2=tf.Variable(tf.constant(2.0,shape=[1]),name='v2')
result=v1+v2

saver=tf.train.Saver()

with tf.Session() as sess:
    
    #加载已保存的模型，并从已保存模型中的变量值计算加法
    saver.restore(sess,'model/1/model1.ckpt')
    print(sess.run(result))
    
#模型计算图不能更改，更改了就不能运行，只少了初始化变量，变量值从保存中抽取

INFO:tensorflow:Restoring parameters from model/1/model1.ckpt
[ 3.]


以上默认保存和加载TensorFlow计算图上定义的全部变量。如需要选择保存或加载部分变量，在声明tf.train.Saver类时可以提供一个列表指定需要保存或加载的变量。例如saver=tf.train.Saver([v1])构建声明，那么只会保存或加载变量v1。tf.train.Saver也支持在保存或加载时给变量命名。

In [2]:
#这里声明的变量名称和已保存模型中的变量名称不同
v_1=tf.Variable(tf.constant(3.0,shape=[1]),name='v_1')
v_2=tf.Variable(tf.constant(2.0,shape=[1]),name='v_2')
result=v_1+v_2

#如果直接使用tf.train.Saver()加载模型会找不到变量而报错
#使用字典重命名变量就能加载原来的模型。

saver=tf.train.Saver({'v1':v_1,'v2':v_2})
with tf.Session() as sess:
    
    saver.restore(sess,'model/1/model1.ckpt')
    print(sess.run(result))

INFO:tensorflow:Restoring parameters from model/1/model1.ckpt
[ 3.]


In [1]:
#保存滑动平均模型的案例

import tensorflow as tf

v=tf.Variable(0,dtype=tf.float32,name='v')

#在没有声明滑动平均模型时只有一个变量v，下面语句会输出“v:0”
for variables in tf.global_variables():
    print(variables.name)
    
ema=tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op=ema.apply(tf.all_variables())

#在声明滑动平均模型后，TensorFlow会自动生成一个影子变量
for variables in tf.global_variables():
    print(variables.name)
    
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.assign(v,10))
    sess.run(maintain_averages_op)
    
    #保存时，tf会将v和v/ExponentialMovingAverage两个变量都存下来
    saver.save(sess,'model/test/movingaverage/ema.ckpt')
    print(sess.run([v, ema.average(v)]))

v:0
Instructions for updating:
Please use tf.global_variables instead.
v:0
v/ExponentialMovingAverage:0
[10.0, 0.099999905]


In [2]:
#以下展示通过重命名直接读取变量的滑动平均值
import tensorflow as tf
v=tf.Variable(0, dtype=tf.float32,name='v')

#通过变量重命名将原来变量v的滑动平均值直接赋值给v
saver=tf.train.Saver({'v/ExponentialMovingAverage':v})
with tf.Session() as sess:
    saver.restore(sess, 'model/test/movingaverage/ema.ckpt')
    print(sess.run(v))

INFO:tensorflow:Restoring parameters from model/test/movingaverage/ema.ckpt
0.0999999


In [1]:
#为了方便加载时重命名滑动平均值，tf.train.ExponentialMovingAverage提供了
#variables_to_restore函数生成tf.train.Saver所需要的变量重命名字典。

import tensorflow as tf

v=tf.Variable(0, dtype=tf.float32,name='v')
ema=tf.train.ExponentialMovingAverage(0.99)

#通过variables_to_restore函数可以直接生成上面代码中提供的字典{'v/ExponentialMovingAverage':v}
print(ema.variables_to_restore())

saver=tf.train.Saver(ema.variables_to_restore())
with tf.Session() as sess:
    saver.restore(sess,'model/test/movingaverage/ema.ckpt')
    print(sess.run(v))


{'v/ExponentialMovingAverage': <tf.Variable 'v:0' shape=() dtype=float32_ref>}
INFO:tensorflow:Restoring parameters from model/test/movingaverage/ema.ckpt
0.0999999


tf提供了convert_to_constants函数，该函数可以将计算图中的变量及其取值通过常量的方式保存，整个计算图和变量可以统一放在一个文件中。

In [3]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

v1=tf.Variable(tf.constant(2.0, shape=[1]),name='v1')
v2=tf.Variable(tf.constant(8.0, shape=[1]),name='v2')
result = v1 + v2

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    #导出当前计算图的GraphDef部分，只需要这一部分就能完成从输入层到输出层的计算过程
    graph_def=tf.get_default_graph().as_graph_def()
    
    #将图中的变量及其取值转化为常量，同时将图中不必要的结点去掉。一些系统运算也会被转化为计算图中的结点（比如变量初始化操作）。
    #如果只关心程序中定义的某些计算，那么和这些计算无关的结点就没有必要导出并保存。
    #以下最后一个参数['add']给出了需要保存的结点名称。add结点是上面定义的求幂运算。
    output_graph_def=graph_util.convert_variables_to_constants(sess, graph_def,['add'] )
    
    #将导出的模型存入文件
    with tf.gfile.GFile('./model/test/1/combined_model.pb','wb') as f:
        f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


In [7]:
#以下可以直接计算定义的加法运算结果，当只需要计算图某个结点的取值时，该方法很方便

import tensorflow as tf
from tensorflow.python.platform import gfile

with tf.Session() as sess:
    model_filename='./model/test/1/combined_model.pb'
    
    #读取保存的模型文件，并将文件解析成对应的GraphDef Protocol Buffer。
    with gfile.FastGFile(model_filename,'rb') as f:
        graph_def=tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    #将graph_def中保存的图加载到当前的图中。return_elements=['add:0']给出了返回的张量名称。在保存的时候给出的是计算结点的名称，
    #所以为“add”。在加载的时候给出的张量的名称，所以是add:0。
    result=tf.import_graph_def(graph_def, return_elements=['add:0'])
    print(sess.run(result))

[array([ 10.], dtype=float32)]
